# Migration, Remittances, and Monetization of Farm Labor in Subsistence Sending Areas

This notebook documents for posterity the code used to clean and format data and generate models for the following paper:

* Hull, J. R. (2007). Migration, Remittances and Monetization of Farm Labor in Subsistence Sending Areas. Asian and Pacific Migration Journal, 16(4), 451–484. https://doi.org/10.1177/011719680701600402

This code is publicly shared in keeping with my commitment to transparency and reproducibility in social science research. <a href=https://www.icpsr.umich.edu/web/DSDR/studies/4402>Data to accompany this code</a> were confidential and used by permission with approval from the Institutional Review Board of my University at the time. For replication or reproduction attempts, the data must now be obtained through the UofM ICPSR's Data Sharing for Demographic Research program. More information is available here: https://www.icpsr.umich.edu/web/pages/DSDR/index.html

The code is presented un-edited with occassional notes or clarifications. 

In [ ]:
********************************************************************
**     Program Name: /home/jrhull/ma/mig00_01.sas
**     Programmer: james r. hull
**     Start Date: 2005 June 19
**     Purpose:
**        1.) Create variables needed to construct migration variable
**        2.) Produce basic descriptive statistics for these variables
**     Input Data:
**        1.) '/nangrong/data_sas/2000/current/indiv00.03'
**        2.) '/nangrong/data_sas/2000/current/hh00.04'
**        3.) '/trainee/jrhull/ma/help0001.xpt'
**     Output Data:
**        1.) '/trainee/jrhull/ma/help0002.xpt'
**
*********************************************************************;

*--------------*
*   Options    *
*--------------*;

options linesize=80 pagesize=60;

title1 'Descriptive analysis: 2000 migration data';

*------------------*
*  Data Libraries  *
*------------------*;

libname in00_1 xport '/nangrong/data_sas/2000/current/hh00.04';
libname in00_2 xport '/nangrong/data_sas/2000/current/indiv00.03';
libname in00_3 xport '/trainee/jrhull/ma/help0001.xpt';

libname out00_1 xport '/trainee/jrhull/ma/help0002.xpt';

*-------------------------*
*  Merge INDIV00 to HH00  *
*-------------------------*;

***Note: this becomes an individual-level file***;

data work00_1 (drop=i);
     merge in00_1.hh00(keep=HHID00 INTMNTH1-INTMNTH6 INTDAY1-INTDAY6 INTRES1-INTRES6 in=a)
           in00_2.indiv00(keep=HHID00 CEP00 X1 X4 X25 X26 X28 X29 CODE2 X7D X7M X7Y in=b);
     by HHID00;

     if (a=1 and b=1) and (X1=3);

*** Create MONTH and DAY ***;

     array iv1 {6} INTRES1-INTRES6;
     array iv2 {6} INTDAY1-INTDAY6;
     array iv3 {6} INTMNTH1-INTMNTH6;

     do i=1 to 6;
        if iv1{i}=1
           then do;
                  DAY=iv2{i};
                  MONTH=iv3{i};
                end;
     end;

     YEAR=2000;

*** Create IDATE, HDATE DDATE ***;

      if MONTH in (99,.) or DAY in (99,.) then IDATE=.;
         else IDATE=MDY(MONTH,DAY,YEAR);

      HDATE=MDY(10,1,1999);

      if IDATE=. or HDATE=. then DDATE=.;
         else DDATE=IDATE-HDATE;

*** Transform DDATE to months ***;

      if DDATE=. then DMONTH=.;
         else DMONTH=round(DDATE/30);

*** Create # Days Away ***;

      if (X7D=99 or X7M=99 or X7Y=99) or (X7M=. and X7Y=.) then DAYSGONE=99999;
         else if (X7D=. and X7Y ne .) then DAYSGONE=(X7Y*365);
         else DAYSGONE=X7D+(X7M*30)+(X7Y*365);

*** Round # days gone to months ***;

      if X7D=99 then MOROUND=9;
         else if X7D=. then MOROUND=.;
         else if X7D<16 then MOROUND=0;
         else if X7D>=16 then MOROUND=1;

*** Create variable # months gone ***;

      if (X7D=99 or X7M=99 or X7Y=99) or (X7M=. and X7Y=.) then MOGONE=999;
         else if (X7D=. and X7Y ne .) then MOGONE=(X7Y*12);
         else MOGONE=MOROUND+X7M+(X7Y*12);

*** Create variable # years gone ***;   /* This was created especially for the cross-tab with X25 & X28 below */

     if MOGONE < 12 then YRGONE=0;
        else if MOGONE > 11 and MOGONE < 24 then YRGONE=1;
        else if MOGONE > 23 and MOGONE < 36 then YRGONE=2;
        else if MOGONE > 35 and MOGONE < 48 then YRGONE=3;
        else if MOGONE > 47 and MOGONE < 60 then YRGONE=4;
        else if MOGONE > 59 and MOGONE < 72 then YRGONE=5;
        else if MOGONE > 71 and MOGONE < 84 then YRGONE=6;
        else if MOGONE > 83 and MOGONE < 96 then YRGONE=7;
        else if MOGONE > 95 and MOGONE < 108 then YRGONE=8;
        else if MOGONE > 107 and MOGONE < 120 then YRGONE=9;
        else if MOGONE > 119 and MOGONE < 132 then YRGONE=10;
        else if MOGONE > 131 and MOGONE < 144 then YRGONE=11;
        else if MOGONE > 143 and MOGONE < 156 then YRGONE=12;
        else if MOGONE > 155 and MOGONE < 168 then YRGONE=13;
        else if MOGONE > 167 and MOGONE < 999 then YRGONE=14;
        else if MOGONE in (999) then YRGONE=99;

*** Compare Months gone to time since 10/1/1999 ***;

      if MOGONE=999 then RICEMIG=9;
         else if (MOGONE > DMONTH and MOGONE < 72) then RICEMIG=1;
         else RICEMIG=0;

*** A second variable usin 3 years as the cut-off, not 6 years ***;

       if MOGONE=999 then RICEMIG2=9;
         else if (MOGONE > DMONTH and MOGONE < 36) then RICEMIG2=1;
         else RICEMIG2=0;

*** Create numeric equivalents (means) of remittances for aggregation ***;

      if X26=1 then REMAMT=500;
         else if X26=2 then REMAMT=2000;
         else if X26=3 then REMAMT=4000;
         else if X26=4 then REMAMT=7500;
         else if X26=5 then REMAMT=15000;
         else if X26=6 then REMAMT=30000;
         else if X26=7 then REMAMT=40000;
         else REMAMT=0;

      if X26 in (.,9) then X26=0;          /* These could be used to look at frequencies - need to be merged to final dataset */

      if X29=1 then SNDAMT=500;
         else if X29=2 then SNDAMT=2000;
         else if X29=3 then SNDAMT=4000;
         else if X29=4 then SNDAMT=7500;
         else if X29=5 then SNDAMT=15000;
         else if X29=6 then SNDAMT=30000;
         else if X29=7 then SNDAMT=40000;
         else SNDAMT=0;

      if X29 in (.,9) then X29=0;          /* These could be used to look at frequencies - need to be merged to final dataset */

run;

*** Frequency ***;

proc freq data=work00_1;
     /* tables DAY MONTH YEAR IDATE HDATE DDATE DMONTH X7D X7M X7Y DAYSGONE MOGONE RICEMIG;*/
     tables YRGONE*(X25 X28);
run;

*** Print ***;

/* proc print data=work00_1(obs=500);
     id HHID00 CEP00;
     var X7D MOROUND X7M X7Y DMONTH MOGONE RICEMIG;
run; */

data work00_2;                                                                       /* Produces a HH-level data-file */
     set work00_1 (keep=HHID00 X4 X25 X28 X26 X29 REMAMT SNDAMT RICEMIG RICEMIG2);

     by HHID00;

     keep HHID00 NUMMIGM NUMMIGF NUMMIGT NUMMIGT2 RECMIG RECMIG2
          MISSMIG MISSMIG2 NUMRRCD2 NUMRRCD3 NUMRSND2 NUMRSND3 NUMREMIT NUMREMSD
          REM_ND2 REM_ND3 SREM_ND2 SREM_ND3 TOTRRCD2 TOTRRCD3 TOTRSND2 TOTRSND3 MIGREM_Y MIGREM_N;

     retain NUMMIGM NUMMIGF NUMMIGT NUMMIGT2 RECMIG RECMIG2
            MISSMIG MISSMIG2 NUMRRCD2 NUMRRCD3 NUMRSND2 NUMRSND3 NUMREMIT NUMREMSD
            REM_ND2 REM_ND3 SREM_ND2 SREM_ND3 TOTRRCD2 TOTRRCD3 TOTRSND2 TOTRSND3 MIGREM_Y MIGREM_N;

     if first.HHID00 then do;
                            NUMMIGT=0;
                            NUMMIGT2=0;
                            RECMIG=0;
                            RECMIG2=0;
                            MISSMIG=0;
                            MISSMIG2=0;
                            NUMMIGM=0;
                            NUMMIGF=0;
                            NUMREMIT=0;
                            NUMREMSD=0;
                            NUMRRCD2=0;
                            NUMRRCD3=0;
                            NUMRSND2=0;
                            NUMRSND3=0;
                            REM_ND2=0;
                            REM_ND3=0;
                            SREM_ND2=0;
                            SREM_ND3=0;
                            TOTRRCD2=0;
                            TOTRRCD3=0;
                            TOTRSND2=0;
                            TOTRSND3=0;
                            MIGREM_Y=0;
                            MIGREM_N=0;
                          end;

     if X25=1 then NUMREMIT=NUMREMIT+1;
     if X28=1 then NUMREMSD=NUMREMSD+1;

     if RICEMIG=1 and X25=1 then NUMRRCD2=NUMRRCD2+1;
        else if RICEMIG=1 and X25=9 then REM_ND2=1;

     if RICEMIG=1 and X28=1 then NUMRSND2=NUMRSND2+1;
        else if RICEMIG=1 and X28=9 then SREM_ND2=1;

     if RICEMIG2=1 and X25=1 then NUMRRCD3=NUMRRCD3+1;
        else if RICEMIG2=1 and X25=9 then REM_ND3=1;

     if RICEMIG2=1 and X28=1 then NUMRSND3=NUMRSND3+1;
        else if RICEMIG2=1 and X28=9 then SREM_ND3=1;

     if RICEMIG=1 and X4=1 then NUMMIGM=NUMMIGM+1;
        else if RICEMIG=1 and X4=2 then NUMMIGF=NUMMIGF+1;

     if RICEMIG=1 then NUMMIGT=NUMMIGT+1;
        else if RICEMIG=0 then RECMIG=RECMIG+1;
        else if RICEMIG=9 then MISSMIG=MISSMIG+1;

     if RICEMIG2=1 then NUMMIGT2=NUMMIGT2+1;
        else if RICEMIG2=0 then RECMIG2=RECMIG2+1;
        else if RICEMIG2=9 then MISSMIG2=MISSMIG2+1;

     if RICEMIG=1 then TOTRRCD2=TOTRRCD2+REMAMT;

     if RICEMIG=1 then TOTRSND2=TOTRSND2+SNDAMT;

     if RICEMIG2=1 then TOTRRCD3=TOTRRCD3+REMAMT;

     if RICEMIG2=1 then TOTRSND3=TOTRSND3+SNDAMT;

     if RICEMIG=1 and X25=1 then MIGREM_Y=MIGREM_Y+1;
        else if RICEMIG=1 then MIGREM_N=MIGREM_N+1;

     if last.HHID00 then output;
run;

proc freq data=work00_2;
     /* tables NUMMIGT; */
     tables NUMREMIT*NUMRRCD2 NUMREMSD*NUMRSND2;
run;

data work00_3 noricefile;                            /* Merging HHs with migrants to all HHs */
     merge work00_2 (in=a)
           in00_3.help0001 (in=b);
     by HHID00;
     if a=0 and b=1 then do;
                            NUMMIGM=0;
                            NUMMIGF=0;
                            NUMMIGT=0;
                            NUMMIGT2=0;
                            MISSMIG=0;
                            MISSMIG2=0;
                            RECMIG=0;
                            RECMIG2=0;
                            NUMREMIT=0;
                            NUMREMSD=0;
                            NUMRRCD2=0;
                            NUMRRCD3=0;
                            NUMRSND2=0;
                            NUMRSND3=0;
                            REM_ND2=0;
                            REM_ND3=0;
                            SREM_ND2=0;
                            SREM_ND3=0;
                            TOTRRCD2=0;
                            TOTRRCD3=0;
                            TOTRSND2=0;
                            TOTRSND3=0;
                            MIGREM_Y=0;
                            MIGREM_N=0;
                         end;

     /* if MISSMIG > 0 then NUMMIG=.;    */  /*This line forces a stricter treatment of missing data */
     if b=1 then output work00_3;
     if a=1 and b=0 then output noricefile;

run;

proc contents data=work00_3 varnum;
run;

proc freq data=work00_3;
     title1 'Number of Migrants per HH';
     tables NUMMIGT NUMMIGM NUMMIGF;
     tables NUMMIGT*HELPDV2 / CHISQ;
     tables NUMMIGT2*HELPDV2 / CHISQ;
     /* tables TOTRRCD2*NUMRRCD2; */
     /* tables TOTRSND2*NUMRSND2; */
run;

proc corr data=work00_2;
     var TOTRRCD2 NUMRRCD2;
     var TOTRSND2 NUMRSND2;
     var TOTRRCD3 NUMRRCD3;
     var TOTRSND3 NUMRSND3;
run;

data out00_1.help0002;
     set work00_3;
run;

proc datasets;
     delete work00_1 work00_2 noricefile;
run;


In [ ]:
*********************************************************************
**     Program Name: /home/jrhull/ma/mig00_02.sas
**     Programmer: james r. hull
**     Start Date: 2005 July 12
**     Purpose:
**        1.) Create control variables for analysis
**        2.)
**     Input Data:
**        1.) '/nangrong/data_sas/2000/current/hh00.04'
**        2.) '/nangrong/data_sas/2000/current/indiv00.03'
**        3.) '/nangrong/data_sas/2000/current/comm00.02'
**        4.) '/trainee/jrhull/ma/help0002.xpt'
**        5.) '/nangrong/data_sas/2000/current/plots00.02'
**        6.) '/trainee/jrhull/ma/wealth00.xpt'
**     Output Data:
**        1.) '/trainee/jrhull/ma/help0003.xpt'
**        2.) '/trainee/jrhull/ma/merge00.xpt'
**
*********************************************************************;

*--------------*
*   Options    *
*--------------*;

options nocenter linesize=80 pagesize=60;

title1 'Creating Control Variables: 2000 migration data';

*------------------*
*  Data Libraries  *
*------------------*;

libname in00_1 xport '/nangrong/data_sas/2000/current/hh00.04';       /* Sorted by HHID00 */
libname in00_2 xport '/nangrong/data_sas/2000/current/indiv00.03';    /* S.B. HHID00 CEP00 */
libname in00_3 xport '/nangrong/data_sas/2000/current/comm00.02';     /* S.B. VILL00 */
libname in00_4 xport '/trainee/jrhull/ma/help0002.xpt';               /* S.B. HHID00 */
libname in00_5 xport '/nangrong/data_sas/2000/current/plots00.02';    /* S.B. HHID00 PLANG00 */
libname in00_6 xport '/trainee/jrhull/ma/wealth00.xpt';

libname out00_1 xport '/trainee/jrhull/ma/help0003.xpt';              /* S.B. HHID00 */
libname out00_2 xport '/trainee/jrhull/ma/merge00.xpt';

*--------------------------------------------*
*  Assemble all necessary control variables  *
*--------------------------------------------*;


data work00_1 (keep=HHID00 CEP00 X1 AGE X4 X25 X28);
     set in00_2.indiv00 (keep=HHID00 CEP00 X1 X3 X4 X25 X28 CODE2);

     *** Recode specially coded ages to numeric equivalents ***;

        if X3=99 then AGE=0;
           else if X3=. then AGE=0; *** Code 1 missing case to 0 for composite measures ***;
           else AGE=X3;

    *** Remove duplicate code 2 cases - leave destination HH data ***;

    if CODE2 ^in (1);

run;

*** HH-level variable creation ***;

data work00_2 (keep=HHID00 AGETOTAL NUMMEMS NUMMALES NUMFEMS
                    NUMDEPCH NUMDEPEL NUMDEPS M_13_65
                    F_13_65 NUMREMIT NUMREMSD REMITND SREMITND);                /* S.B. HHID00 */
     set work00_1;
     by HHID00;

     keep HHID00 AGETOTAL NUMMEMS NUMMALES NUMFEMS NUMDEPCH
          NUMDEPEL NUMDEPS M_13_65 F_13_65 NUMREMIT NUMREMSD
          REMITND SREMITND;

     retain AGETOTAL NUMMEMS NUMMALES NUMFEMS NUMDEPCH NUMDEPEL
            NUMDEPS M_13_65 F_13_65 NUMREMIT NUMREMSD REMITND SREMITND;

     if first.HHID00 then do;
                            AGETOTAL=0;
                            NUMMEMS=0;
                            NUMMALES=0;
                            NUMFEMS=0;
                            NUMDEPCH=0;
                            NUMDEPEL=0;
                            NUMDEPS=0;
                            M_13_65=0;
                            F_13_65=0;
                            NUMREMIT=0;
                            NUMREMSD=0;
                            REMITND=0;
                            SREMITND=0;
                          end;

     if X1 ^in (0,2,3,9) then do;

                            if ((13 <= AGE <=65) and (X4=1))
                               then M_13_65=M_13_65+1; /*Males 13-65*/
                            if ((13 <= AGE <=65) and (X4=2))
                               then F_13_65=F_13_65+1; /*Females 13-65*/

                            if (AGE < 13) then NUMDEPCH=NUMDEPCH+1;
                            if (AGE > 65) then NUMDEPEL=NUMDEPEL+1;

                            NUMDEPS=NUMDEPCH+NUMDEPEL;

                            AGETOTAL=AGETOTAL+AGE;

                            NUMMEMS=NUMMEMS+1;

                            if X4=1 then NUMMALES=NUMMALES+1;
                               else if X4=2 then NUMFEMS=NUMFEMS+1;

                            if X25=1 then NUMREMIT=NUMREMIT+1;
                               else if X25=9 then REMITND=1;

                            if X28=1 then NUMREMSD=NUMREMSD+1;
                               else if X28=9 then SREMITND=1;

                         end;

     if last.HHID00 then output;

run;


*** add variables VILL00 and VILL94 to collapsed individual-level data ***;

data work00_1b noidvars nodata;                             /* S.B. HHID00 */
     merge work00_1 (in=a)
           in00_1.hh00 (keep= HHID00 VILL00 VILL94 in=b);
     by HHID00;
     if a=1 and b=1 then output work00_1b;
     if a=1 and b=0 then output noidvars;
     if a=0 and b=1 then output nodata;
run;

*** Sort by VILL94 before collapsing data by village ***;

proc sort data=work00_1b out=work00_1c;                     /* S.B. VILL94 */
     by VILL94;
run;

*** village-level variable creation ***;

data work00_3 (keep=VILL94 VILL_WAM VILL_WAF);             /* S.B. VILL94 */
     set work00_1c;
     by VILL94;

     keep VILL94 VILL_WAM VILL_WAF;

     retain VILL_WAM VILL_WAF;

     if first.VILL94 then do;
                            VILL_WAM=0;
                            VILL_WAF=0;
                          end;


     if ((13 <= AGE <=65) and (X4=1)) then VILL_WAM=VILL_WAM+1; /*Males 13-65*/

     if ((13 <= AGE <=65) and (X4=2)) then VILL_WAF=VILL_WAF+1; /*Females 13-65*/

     if last.VILL94 then output;

run;

data work00_4a;                                           /* S.B. HHID00 */
     set in00_5.plots00 (keep=HHID00 PLANG00 X6_20_1 X6_14NGA X6_14RAI X6_14WA X6_15NGA X6_15RAI X6_15WA);

     by HHID00;

     keep HHID00 HH_RAI;

     retain HH_RAI;

     if first.HHID00 then do;
                            HH_RAI=0;
                          end;

     if X6_20_1=1 then do;
                          if X6_14RAI ^in (9999,0,.) then HH_RAI=HH_RAI+X6_14RAI;
                             else if X6_14NGA ^in (99,0,.) then HH_RAI=HH_RAI+(0.25*X6_14NGA);
                             else if X6_14WA ^in (9999,0,.) then HH_RAI=HH_RAI+(0.0025*X6_14WA);
                             else if X6_15RAI ^in (9999,.,0) then HH_RAI=HH_RAI+X6_15RAI;
                             else if X6_15NGA ^in (9999,.,0) then HH_RAI=HH_RAI+(0.25*X6_15NGA);
                             else if X6_15WA ^in (9999,.,0) then HH_RAI=HH_RAI+(0.0025*X6_15WA);
                       end;

     if last.HHID00 then output;

run;

data work00_4b notinhh00;                               /* S.B. HHID00 */
     merge work00_4a (in=a)
           in00_1.hh00 (in=b keep=HHID00 VILL00 VILL94);
     by HHID00;
     if b=1 then output work00_4b;
     if a=1 and b=0 then output notinhh00;
run;

data work00_4c;                                        /* S.B. HHID00 */
     set work00_4b;

     if HH_RAI=. then HH_RAI=0;  /* Treat missing as 0 for village calc's */
run;

*** sort HH-level data by VILL94 before collapsing to village-level ***;

proc sort data=work00_4c out=work00_4d;                /* S.B. VILL94 */
     by VILL94;
run;

data work00_4e;                                        /* S.B. VILL94 */
     set work00_4d;

     by VILL94;

     keep VILL94 VILL_RAI;

     retain VILL_RAI;

     if first.VILL94 then do;
                            VILL_RAI=0;
                          end;

     if HH_RAI ne 999999 then VILL_RAI=VILL_RAI+HH_RAI;
        else VILL_RAI=VILL_RAI;

     if last.VILL94 then output;


run;

*** merge village-level variables ***;

data work00_4f novillrai nohhidvars novilldems;        /* S.B. VILL94 - ALL */
     merge work00_4e (in=a)
           work00_4d (in=b)
           work00_3 (in=c);
     by VILL94;
     if a=1 and b=1 and c=1 then output work00_4f;
     if a=0 then output novillrai;
     if b=0 then output nohhidvars;
     if c=0 then output novilldems;
run;

proc sort data=work00_4f out=work00_4g;                /* S.B. HHID00 */
     by HHID00;
run;

*** Get other HH-level variables needed ***;

data work00_5 (drop=X6_10A: X6_10B: X6_3A:);                                         /* S.B. HHID00 */
     merge in00_1.hh00 (keep=VILL00 HHID00 X6_3A1 X6_3A2
                             X6_3A3 X6_3A4 X6_10A1 X6_10A2 X6_10A3 X6_10B1
                             X6_10B2 X6_10B3 X6_87A1 X6_87A2 X6_87A3
                             CASSAVA HHTYPE00 in=a)
           work00_2 (in=b);
     by HHID00;

     if NUMMEMS ne 0 then do;
                           MEANAGE=AGETOTAL/NUMMEMS;
                          end;
     else MEANAGE=.;

     /*if REMITND=1 then NUMREMIT=.; */ /* These statements force a stricter treatment of missing data */
     /*if SREMITND=1 then NUMREMSD=.; */

     if X6_3A1=0 then SILK=0;
        else if X6_3A1=9 then SILK=.;
        else SILK=1;

     if X6_3A2=0 then SILKWORM=0;
       else if X6_3A2=9 then SILKWORM=.;
       else SILKWORM=1;

     if X6_3A3=0 then CLOTH=0;
       else if X6_3A3=9 then CLOTH=.;
       else CLOTH=1;

     if X6_3A4=0 then CHARCOAL=0;
       else if X6_3A3=9 then CHARCOAL=.;
       else CHARCOAL=1;

     if X6_10A1 in (1,2) then COWS=X6_10B1;
        else if X6_10A1=9 then COWS=.;
        else if X6_10A1=3 then COWS=0;

     if X6_10A2 in (1,2) then BUFFALO=X6_10B2;
        else if X6_10A2=9 then BUFFALO=.;
        else if X6_10A2=3 then BUFFALO=0;

     if X6_10A3 in (1,2) then PIGS=X6_10B3;
        else if X6_10A3=9 then PIGS=.;
        else if X6_10A3=3 then PIGS=0;

     if CASSAVA in (.,2) then CASSAVA=0;
        else if CASSAVA=9 then CASSAVA=1;

     if SILK=1 or SILKWORM=1 or CLOTH=1 then COTTAGE=1;
        else COTTAGE=0;

run;

*** merge HH-level variables ***;

data work00_6a nohhlevelvars nohhrai novillevelvars;                    /* S.B. HHID00 */
     merge work00_5 (in=a)
           work00_4c (in=b)
           work00_4g (in=c);
     by HHID00;
     if a=1 and b=1 then output work00_6a;
     if a=0 then output nohhlevelvars;
     if b=0 then output nohhrai;
     if c=0 then output novillevelvars;
run;


*** merge village-level variables onto HH-level variables ***;

data work00_6b nosurvey;
     merge work00_6a (in=a)
           in00_3.comm00 (in=b keep=VILL00 X45MALE X45FEM X45MHIGH X45FHIGH
                                    X45MTYP X45FTYP);

     by VILL00;

     if a=1 and b=1 then output work00_6b;
     if a=0 and b=1 then output nosurvey;
run;


data work00_7 nodepvar nocontrols;
     merge work00_6b (in=a)
           in00_4.help0002 (in=b);
     if a=1 and b=1 then output work00_7;
     if a=1 and b=0 then output nodepvar;
     if a=0 and b=1 then output nocontrols;

run;


data work00_8 (drop=X6_87A: X45MALE X45FEM CCAT23F CCATALLA CCATOF CCATVF);
     set work00_7 (drop= HELP23C: HELP23F: HELPOC: HELPOE: HELPOF: HELPVC: HELPVE:
                         HELPVF:);

     if X6_87A1=999 then RICE_JAS=999;
        else if X6_87A1=. then RICE_JAS=0;
        else RICE_JAS=X6_87A1*(100/15);          /* This converts from grasops to tang */

     if X6_87A2=999 then RICE_STK=999;
        else if X6_87A2=. then RICE_STK=0;
        else RICE_STK=X6_87A2*(100/15);          /* This converts from grasops to tang */

     if X6_87A3=999 then RICE_OTH=999;
        else if X6_87A3=. then RICE_OTH=0;
        else RICE_OTH=X6_87A3*(100/15);          /* This converts from grasops to tang */

     if RICE_JAS=999 & RICE_STK=999 & RICE_OTH=999 then RICE_YLD=.;
        else if RICE_JAS=999 & RICE_STK ne 999 & RICE_OTH ne 999 then RICE_YLD=RICE_STK+RICE_OTH;
        else if RICE_JAS=999 & RICE_STK=999 & RICE_OTH ne 999 then RICE_YLD=RICE_OTH;
        else if RICE_JAS=999 & RICE_STK ne 999 & RICE_OTH=999 then RICE_YLD=RICE_STK;
        else if RICE_JAS ne 999 & RICE_STK=999 & RICE_OTH ne 999 then RICE_YLD=RICE_JAS+RICE_OTH;
        else if RICE_JAS ne 999 & RICE_STK ne 999 & RICE_OTH=999 then RICE_YLD=RICE_JAS+RICE_STK;
        else if RICE_JAS ne 999 & RICE_STK=999 & RICE_OTH=999 then RICE_YLD=RICE_JAS;
             else RICE_YLD=RICE_JAS+RICE_STK+RICE_OTH;


     if NUMMEMS=0 then NUMMEMS=.;    /* There are two HHS who had no living members in 00 */

     if HELPHH=. then HELPHH=0;
     if HELP23B=. then HELP23B=0;
     if HELPVB=. then HELPVB=0;
     if HELPOB=. then HELPOB=0;

     TOTHELP=HELP23B+HELPVB+HELPOB;
     TOTHELP2=HELPHH+HELP23B+HELPVB+HELPOB;

     VILL1365=VILL_WAM+VILL_WAF;  /*Sums working age males and females in village (collinearity problems) */

     if X45MALE=. then V_HELPM=0;
        else V_HELPM=X45MALE;
     if X45FEM=. then V_HELPF=0;
        else V_HELPF=X45FEM;

     label AGETOTAL='sum ages of HH members';
     label BUFFALO= '# buffalo raised by HH';
     label CHARCOAL= 'HH makes charcoal: 0-no 1-yes';
     label CLOTH= 'HH makes cloth: 0-no 1-yes';
     label COWS= '# cows raised by HH';
     label COTTAGE= 'HH engages in a cottage industry';
     label F_13_65= '# working age females in HH';
     label HELPDV= 'HH rice harvest strategy measure 1';
     label HELPDV2= 'HH rice harvest strategy measure 2';
     label HH_RAI= '# rai rice paddy used by HH last year';
     label MEANAGE= 'Mean age of HH members';
     label MISSMIG= '# HH migrants of unknown duration';
     label M_13_65= '# working age males in HH';
     label NUMFEMS= '# females in HH';
     label NUMMALES= '# males in HH';
     label NUMMEMS= '# HH members';
     label NUMMIGT= '# HH migrants (left b/f last harvest)';
     label NUMMIGM= '# male HH migrants (b/f last harvest)';
     label NUMMIGF= '# female HH migrants (b/f last harvest)';
     label NUMREMIT= '# HH members remitting money';
     label NUMREMSD= '# HH members receiving money';
     label NUMRRCD2= '# HH mig (left b/f harvest) remit';
     label NUMRSND2= '# HH mig (left b/f harvest) receive';
     label NUMDEPCH= '# HH members under age 13';
     label NUMDEPEL= '# HH members over age 65';
     label NUMDEPS= '#HH members under 13 & over 65';
     label PIGS= '# pigs raised by HH';
     label RECMIG= '# HH migrants (left after last harvest)';
     label SILK= 'HH makes silk: 0-no 1-yes';
     label SILKWORM= 'HH raises silkworm 0-no 1-yes';
     label VILL_RAI= '# rai rice paddy village last yr';
     label VILL_WAF= '# working age females in village';
     label VILL_WAM= '# working age males in village';
     label RICE_YLD= '00: amount of rice yield - tangs (Q87)';
     label V_HELPM= '00: males hired for labor (Q45)';
     label V_HELPF= '00: females hired for labor (Q45)';
     label VILL1365= '# working age adults in village';
     label TOTHELP= 'Total # helpers all sources';
     label TOTHELP2= 'Total # helpers all sources (plus HH)';
     label NUMMIGT2= '# HH mig (b/f harvest) 3-yr window';
     label RECMIG2= '# HH mig (after harvest) 3-yr window';
     label MISSMIG2= '# HH mig (left unknown) 3-yr window';
     label NUMRRCD3= '# HH remit (b/f harvest) 3-yr win';
     label NURRSND3= '# HH receive (b/f harvest) 3-yr win';

     rename X45FHIGH=V_WAGEFH X45MHIGH=V_WAGEMH X45FTYP=V_WAGEFT X45MTYP=V_WAGEMT;

run;

proc datasets;
     delete work00_1: work00_2 work00_3 work00_4: work00_5 work00_6: work00_7
            nodata noidvars novillrai nohhidvars novilldems nodepvar nocontrols
            nosurvey nohhrai nohhlevelvars novillevelvars notinhh00;
run;

 *** Add in wealth index variables ***;

data work00_9 wealthonly;
     merge work00_8 (in=a)
           in00_6.wealth00 (in=b);
     by hhid00;

     if a=1 then output work00_9;
     if a=1 and b=0 then do;
                            LOWEST=.;
                            MIDDLE=.;
                            HIGHEST=.;
                         end;

     if a=0 and b=1 then output wealthonly;
run;

data work00_0;
     set work00_9;

     label LOWEST= 'HH is in lowest HH Assets PCA grouping';
     label MIDDLE= 'HH is in middle HH Assets PCA grouping';
     label HIGHEST= 'HH is in highest HH Assets PCA grouping';
run;

*** Add in count of code 2 migrants per household ***;

data work00_1;
     set in00_2.indiv00 (keep=HHID00 X1);

     by HHID00;

     keep HHID00 CODETWO;

     retain CODETWO;

     if first.HHID00 then do;
                            CODETWO=0;
                          end;

     if X1=2 then CODETWO=CODETWO+1;

     if last.HHID00 then output;
run;

data work00_2;
     merge work00_1 (in=a)
           work00_0 (in=b);
     by HHID00;

     label CODETWO= '# Former HH members living in village';

     if a=1 and b=1 then output;
run;

proc contents data=work00_2;
run;

data out00_1.help0003;
     set work00_2;
     if HHTYPE00 in (1,3);   *** Remove NEW HH's from final file ***;
run;

*** Create a file to use in match-merging the 1994 and 2000 data ***;

data out00_2.merge00;
     set work00_2 (keep=VILL94 HHID94 VILL00 HHID00 HELPDV HELPDV2 NUMMIGT NUMRRCD2 NUMRSND2 HHTYPE00 CODETWO
                        M_13_65 F_13_65 NUMDEPS MEANAGE CASSAVA COTTAGE PIGS COWS
                        CHARCOAL LOWEST MIDDLE HIGHEST VILL1365 VILL_RAI);
    rename HELPDV=HELPD100 HELPDV2=HELPD200 NUMMIGT=NUMMIG00 NUMRRCD2=NUMRCD00 NUMRSND2=NUMSND00
           CODETWO=CODE2_00 M_13_65=M1365_00  F_13_65=F1365_00 NUMDEPS=NUMDEP00 MEANAGE=MNAGE_00
           CASSAVA=CASS_00 COTTAGE=COTT_00 PIGS=PIGS_00 COWS=COWS00 CHARCOAL=CHAR_00
           LOWEST=LOW_00 MIDDLE=MID_00 HIGHEST=HIGH_00 VILL1365=V1365_00 VILL_RAI=VRAI_00;

run;


In [ ]:
*********************************************************************
**     Program Name: /home/jrhull/ma/mig00_03.sas
**     Programmer: james r. hull
**     Start Date: 2005 September 19
**     Purpose:
**        1.) Descriptive Analysis
**        2.) Check for bad data and distributions before transfer
**     Input Data:
**        1.) '/trainee/jrhull/ma/help0003.xpt'
**     Output Data:
**        1.) '/trainee/jrhull/ma/help0004.xpt'
**
*********************************************************************;

*--------------*
*   Options    *
*--------------*;

options nocenter linesize=80 pagesize=60;

title1 'Creating Control Variables: 2000 migration data';

*------------------*
*  Data Libraries  *
*------------------*;

libname in00_1 xport '/trainee/jrhull/ma/help0003.xpt';
libname in00_2 xport '/nangrong/data_sas/2000/current/indiv00.03';

*--------------------------------------------------*
* Frequencies, Means, and Std. Dev's for Variables *
*--------------------------------------------------*;

proc freq data=in00_1.help0003 (drop=HHID00 HHID94 HOUSE00 HOUSE84 VILL00 VILL84 VILL94 AGETOTAL MEANAGE);
     tables HELP23A*HELPVA*HELPOA;           *** Remove this line to produce freq's for all variables ***;
     tables V_WAGE:;
     tables NUMMIGT MIGREM_Y MIGREM_N;
run;

*** Generate means for total sample ***;

/* proc means data=in00_1.help0003;
     vars AGETOTAL BUFFALO CASSAVA COWS F_13_65 HELPOB HELPVB HH_RAI M_13_65 MEANAGE NUM:
          LOWEST MIDDLE HIGHEST SILK SILKWORM CLOTH CHARCOAL COTTAGE PIGS MISSMIG RECMIG
          VILL1365 VILL_RAI VILL_WAF VILL_WAM HELP23B HELPOB HELPOH HELPVB HELPVH TOTHELP
          V_WAGE:;
run;   */

/* proc sort data=in00_1.help0003 out=help0003s2;
     by HELPDV2;
run;

proc means data=help0003s2 n mean std fw=4;
     vars AGETOTAL BUFFALO CASSAVA COWS F_13_65 HELPOB HELPVB HH_RAI M_13_65 MEANAGE NUM:
          LOWEST MIDDLE HIGHEST SILK SILKWORM CLOTH CHARCOAL COTTAGE PIGS MISSMIG RECMIG
          VILL1365 VILL_RAI VILL_WAF VILL_WAM HELP23B HELPOB HELPOH HELPVB HELPVH TOTHELP;
     by HELPDV2;
run; */

/* proc sort data=in00_1.help0003 out=help0003s3;
     by HELPDV;
run;

proc means data=help0003s3 n mean std fw=4;
     vars AGETOTAL BUFFALO CASSAVA COWS F_13_65 HELPOB HELPVB HH_RAI M_13_65 MEANAGE NUM:
          LOWEST MIDDLE HIGHEST SILK SILKWORM CLOTH CHARCOAL COTTAGE PIGS MISSMIG RECMIG
          VILL1365 VILL_RAI VILL_WAF VILL_WAM HELP23B HELPOB HELPOH HELPVB HELPVH TOTHELP;
     by HELPDV;
run; */

*** Additional analyses ***;

/* proc freq data=in00_1.help0003;
     tables VILL94*HELPDV VILL94*HELPDV2 VILL94*VILL_MIG/ nofreq nopercent nocol;
     tables CODETWO NUMMIGT;
     tables (NUMRRCD2 NUMRSND2)*NUMMIGT;
     tables HELPDV2;
     tables (NUMRRCD2 NUMRSND2)*HELPDV2;
     tables NUMMIGT*HELPDV;
run; */

proc freq data=in00_2.indiv00;
     tables HHTYPE00*X1;
run;


In [ ]:
*********************************************************************
**     Program Name: /home/jrhull/ma/mig00_04.sas
**     Programmer: james r. hull
**     Start Date: 2006 February 15
**     Purpose:
**        1.) Additional Village-Level Analysis
**
**     Input Data:
**        1.) '/trainee/jrhull/ma/help0003.xpt'
**     Output Data:
**        1.)
**
*********************************************************************;

*--------------*
*   Options    *
*--------------*;

options nocenter linesize=80 pagesize=60;

title1 'Additional Analysis: Village-Level Relationship between Migration and Yields';

*------------------*
*  Data Libraries  *
*------------------*;

libname in00_1 xport '/trainee/jrhull/ma/help0003.xpt';

proc sort data=in00_1.help0003 out=work00_0;
     by VILL84;
run;

data work00_1 (keep=VILL84 VILL_MIG VILL_YLD VILL_ADS VILL_LND VILL_HH VILL_RCD VILL_SND);
     set work00_0;
     by VILL84;

     keep VILL84 VILL_MIG VILL_YLD VILL_ADS VILL_LND VILL_HH VILL_RCD VILL_SND;

     retain VILL_MIG VILL_YLD VILL_ADS VILL_LND VILL_HH VILL_RCD VILL_SND;

     if first.VILL84 then do;                                /* Change between VILL84 and VILL94 */
                            VILL_MIG=0;
                            VILL_YLD=0;
                            VILL_LND=VILL_RAI;
                            VILL_ADS=VILL1365;
                            VILL_HH=0;
                            VILL_RCD=0;
                            VILL_SND=0;
                          end;

     if NUMMIGT ne . then VILL_MIG=VILL_MIG+NUMMIGT;    /* Number of pre-harvest migrants at village level */
        else VIL_MIG=VIL_MIG;

     if RICE_YLD ne . then VILL_YLD=VILL_YLD+RICE_YLD;    /* Total rice yield at village level */
        else VILL_YLD=VILL_YLD;

     if NUMRRCD2 ne . then VILL_RCD=VILL_RCD+NUMRRCD2;
        else VILL_RCD=VILL_RCD;

     if NUMRSND2 ne . then VILL_SND=VILL_SND+NUMRSND2;
        else VILL_SND=VILL_SND;

     VILL_HH=VILL_HH+1;

     if last.VILL84 then output;

run;

data work00_2;
     set work00_1;
     MIG_RAT=VILL_MIG/VILL_ADS;
     VILL_MHH=VILL_MIG/VILL_HH;
run;

proc print data=work00_2;
     by VILL84;
     var VILL_MIG VILL_YLD VILL_ADS VILL_LND MIG_RAT VILL_HH VILL_MHH VILL_RCD VILL_SND;
run;

proc corr data=work00_2;
     var VILL_YLD VILL_MIG VILL_ADS VILL_LND MIG_RAT VILL_HH VILL_MHH VILL_RCD VILL_SND;
run;

/* proc means data=work00_2;
     var VILL_MIG VILL_YLD VILL_ADS VILL_LND MIG_RAT;
run;

proc corr data=work00_2;
     var VILL_YLD VILL_MIG;
run;

proc reg data=work00_2;
     model VILL_YLD=VILL_MIG;
run;

proc reg data=work00_2;
     model VILL_YLD=VILL_MIG VILL_ADS VILL_LND;
run;

proc reg data=work00_2;
     model VILL_YLD=MIG_RAT;
run;

proc reg data=work00_2;
     model VILL_YLD=MIG_RAT VILL_LND;
run; */


In [ ]:
*********************************************************************
**     Program Name: /home/jrhull/ma/des00_01.sas
**     Programmer: james r. hull
**     Start Date: 2005 April 7
**     Purpose:
**        1.) Create dataset needed to examine descriptive statistics
**            on dependent variable (HH rice strategy) for the 2000
**            data set
**     Input Data:
**        1.) /nangrong/data_sas/2000/current/hh00.04
**        2.) /nangrong/data_sas/2000/current/plots00.02
**
**     Output Data:
**        1.) /trainee/jrhull/ma/help0001.xpt
**
*********************************************************************;

***************
**  Options  **
***************;

options nocenter linesize=80 pagesize=60;

title1 'Program to create HH-level rice harvest variables: 2000';

**********************
**  Data Libraries  **
**********************;

libname in00_1 xport '/nangrong/data_sas/2000/current/hh00.04';
libname in00_2 xport '/nangrong/data_sas/2000/current/plots00.02';

libname out00_1 xport '/trainee/jrhull/ma/help0001.xpt';

libname extra_1 xport '/nangrong/data_sas/1994/current/hh94.03';

******************************
**  Create Working Dataset  **
******************************;

***********************************************************
**  Variables initially in Work1 dataset:
**
**
**   vill00   = 2000 identifiers
**   house00
**   cep00
**   hhid00
**
**   vill94   = 1994 identifiers
**   lekti94
**   cep94
**   hhid94
**
**   vill84 = 1984 identifiers
**   house84
**   cep84
**   hhid84
**
**   rice = Interviewer checks if HH planted rice (see 6.20)
**
**   x6_83 = Last year, how many people in your household
**          helped with the harvesting of rice?
**
**   x6_84 = About harvesting rice last year, were there
**          people in your household in 1984 ad/or 1994
**          that do not currently live in your house who
**          helped with the rice harvest?
**   x6_84c1-x6_84c7 = 2000 CEP Number
**   x6_84w1-x6_84w7 = Type of labor (Hired/Wage, Free,
**         Labor exchange, N/A, missing)
**
**   x6_85 = Did anyone from this village help to harvest
**         rice in the last year?
**   x6_85h1-x6_85h13 = 2000 Household ID
**   x6_85n1-x6_85n13 = Number of people who helped
**   x6_85w1-x6_85w13 = Type of labor?
**
**   x6_86 = Did anyone from another village come to
**         help harvest rice in the last year?
**   x6_86l1-x6_86l10 = Location information
**   x6_86n1-x6_86n10 = Number persons who helped
**   x6_86w1-x6_86w10 = Type of labor (hired, free,
**           labor exchange, N/A, Missing/Don't Know)
**
**   x6_87a1 = number grasops of Jasmine rice harvested
**   x6_87a2 = number of grasops of sticky rice harvested
**   x6_87a3 = number of grasops of other rice harvested
**
************************************************************;

data work00_1;
     set in00_1.hh00;
     keep VILL00 HOUSE00 HHID00 VILL94 HHID94
        VILL84 HOUSE84 RICE X6_83
        X6_84 X6_84C: X6_84W: X6_85 X6_85H: X6_85N: X6_85W:
        X6_86 X6_86l: X6_86N: X6_86W:;
     rename X6_83=HELPHH X6_84=HELP23A X6_84C1=HELP23C1
        X6_84C2=HELP23C2 X6_84C3=HELP23C3 X6_84C4=HELP23C4
        X6_84C5=HELP23C5 X6_84C6=HELP23C6 X6_84C7=HELP23C7
        X6_84W1=HELP23F1 X6_84W2=HELP23F2 X6_84W3=HELP23F3
        X6_84W4=HELP23F4 X6_84W5=HELP23F5 X6_84W6=HELP23F6
        X6_84W7=HELP23F7 X6_85=HELPVA X6_85H1=HELPVC1
        X6_85H2=HELPVC2 X6_85H3=HELPVC3 X6_85H4=HELPVC4
        X6_85H5=HELPVC5 X6_85H6=HELPVC6 X6_85H7=HELPVC7
        X6_85H8=HELPVC8 X6_85H9=HELPVC9 X6_85H10=HELPVC10
        X6_85H11=HELPVC11 X6_85H12=HELPVC12 X6_85H13=HELPVC13
        X6_85N1=HELPVE1 X6_85N2=HELPVE2 X6_85N3=HELPVE3
        X6_85N4=HELPVE4 X6_85N5=HELPVE5 X6_85N6=HELPVE6
        X6_85N7=HELPVE7 X6_85N8=HELPVE8 X6_85N9=HELPVE9
        X6_85N10=HELPVE10 X6_85N11=HELPVE11 X6_85N12=HELPVE12
        X6_85N13=HELPVE13 X6_85W1=HELPVF1 X6_85W2=HELPVF2
        X6_85W3=HELPVF3 X6_85W4=HELPVF4 X6_85W5=HELPVF5
        X6_85W6=HELPVF6 X6_85W7=HELPVF7 X6_85W8=HELPVF8
        X6_85W9=HELPVF9 X6_85W10=HELPVF10 X6_85W11=HELPVF11
        X6_85W12=HELPVF12 X6_85W13=HELPVF13 X6_86=HELPOA
        X6_86L1=HELPOC1 X6_86L2=HELPOC2 X6_86L3=HELPOC3
        X6_86L4=HELPOC4 X6_86L5=HELPOC5 X6_86L6=HELPOC6
        X6_86L7=HELPOC7 X6_86L8=HELPOC8 X6_86L9=HELPOC9
        X6_86L10=HELPOC10 X6_86N1=HELPOE1 X6_86N2=HELPOE2
        X6_86N3=HELPOE3 X6_86N4=HELPOE4 X6_86N5=HELPOE5
        X6_86N6=HELPOE6 X6_86N7=HELPOE7 X6_86N8=HELPOE8
        X6_86N9=HELPOE9 X6_86N10=HELPOE10 X6_86W1=HELPOF1
        X6_86W2=HELPOF2 X6_86W3=HELPOF3 X6_86W4=HELPOF4
        X6_86W5=HELPOF5 X6_86W6=HELPOF6 X6_86W7=HELPOF7
        X6_86W8=HELPOF8 X6_86W9=HELPOF9 X6_86W10=HELPOF10;

run;

data work00_2 (drop= i j k HELP23B1-HELP23B7 HELPVT1-HELPVT13 HELPOT1-HELPOT10);
     set work00_1;

     array b(1:7) HELP23B1-HELP23B7;
     array c(1:7) HELP23C1-HELP23C7;
     array f(1:7) HELP23F1-HELP23F7;

     array vc(1:13) HELPVC1-HELPVC13;
     array ve(1:13) HELPVE1-HELPVE13;
     array vt(1:13) HELPVT1-HELPVT13;
     array vf(1:13) HELPVF1-HELPVF13;

     array oc(1:10) HELPOC1-HELPOC10;
     array oe(1:10) HELPOE1-HELPOE10;
     array ot(1:10) HELPOT1-HELPOT10;
     array of(1:10) HELPOF1-HELPOF10;

     do i=1 to 7;
        if c(i) ne '  ' then b(i)=1;
           else b(i)=0;
       if c(i)='99' then c(i)='  ';
        if f(i)=9 then f(i)=.;
     end;

     do j=1 to 13;
        if ve(j)=99 then ve(j)=.;
        if vf(j)=9 then vf(j)=.;
        if ve(j)=. then vt(j)=0;
           else vt(j)=ve(j);
     end;

     do k=1 to 10;
        if oe(k)=99 then oe(k)=.;
        if of(k)=9 then of(k)=.;
        if oe(k)=. then ot(k)=0;
           else ot(k)=oe(k);
     end;

     if RICE=2 then RICE=0;
        else if RICE=. then RICE=0;
        else if RICE=9 then RICE=.;

     if HELPHH=99 then HELPHH=.;

     if HELP23A=2 then HELP23A=0;
        else if HELP23A=9 then HELP23A=0;
        else if HELP23A=. then HELP23A=0;

     if HELPVA=2 then HELPVA=0;
        else if HELPVA=9 then HELPVA=0;
        else if HELPVA=. then HELPVA=0;

     if HELPOA=2 then HELPOA=0;
        else if HELPOA=9 then HELPOA=0;
        else if HELPOA=. then HELPOA=0;

     HELP23B = HELP23B1+HELP23B2+HELP23B3+HELP23B4+HELP23B5+HELP23B6+HELP23B7;
     HELPVB = sum(of HELPVT1-HELPVT13);
     HELPOB = sum(of HELPOT1-HELPOT10);

     if HELP23B = 0 then HELP23B = .;
     if HELPVB = 0 then HELPVB =.;
     if HELPOB = 0 then HELPOB = .;

     label HELP23B = 'Total # of code 2 and 3 helpers';
     label HELPVB = 'Total # of helpers from same village';
     label HELPOB = 'Total # of helpers from other villages';

run;


/* proc contents data=work00_2 varnum;
run; */

/* proc datasets library=work;
     delete work00_1;
run; */

*************************************************************
**  Freqs, means, and sd for all variables in the dataset  **
*************************************************************;

/* proc freq data=work00_2;
     tables RICE HELP23A HELPVA HELPOA /missprint;
run;

proc means data=work00_2 maxdec=2 mean std min max nmiss;
     var HELP23B HELPVB HELPOB;
run; */

******************************************************************
**  Concatenate variables A and F for groups 23, V, and O       **
**  asking about paid labor status into single strings in       **
**  order to examine a frequency distribution of all sequences  **
******************************************************************;

data work00_3 (drop=string1-string3 i); /* concatenate 'A' variables */
    set work00_2;

    length string1-string3  $ 1 CCATALLA $ 3;

    array a (3) HELP23A HELPVA HELPOA;
    array b (3) string1-string3;

    do i=1 to 3;
      if a{i}=1 then b{i}='1';
      else if a{i}=2 then b{i}='2';
      else if a{i}=3 then b{i}='3';
      else b{i}=' ';
    end;

    CCATALLA=string1||string2||string3;


    label CCATALLA="concatenation of all A variables";
run;


/* concatenate labor ?s*/


data work00_4 (drop= stringa1-stringa7 strngb1-strngb13
       strngc1-strngc10 i j k);
    set work00_3;

    length stringa1-stringa7  $ 1 CCAT23F $ 7;
    length strngb1-strngb13  $ 1 CCATVF $ 13;
    length strngc1-strngc10  $ 1 CCATOF $ 10;

    array va (7) HELP23F1-HELP23F7;
    array a (7) stringa1-stringa7;
    array vb (13) HELPVF1-HELPVF13;
    array b (13) strngb1-strngb13;
    array vc (10) HELPOF1-HELPOF10;
    array c (10) strngc1-strngc10;

    do i=1 to 7;
      if va{i}=1 then a{i}='1';
      else if va{i}=2 then a{i}='2';
      else if va{i}=3 then a{i}='3';
      else a{i}=' ';
    end;

    do j=1 to 13;
      if vb{j}=1 then b{j}='1';
      else if vb{j}=2 then b{j}='2';
      else if vb{j}=3 then b{j}='3';
      else b{j}=' ';
    end;

    do k=1 to 10;
      if vc{k}=1 then c{k}='1';
      else if vc{k}=2 then c{k}='2';
      else if vc{k}=3 then c{k}='3';
      else c{k}=' ';
    end;

    CCAT23F=stringa1||stringa2||stringa3||stringa4||stringa5||
            stringa6||stringa7;
    CCATVF=strngb1||strngb2||strngb3||strngb4||strngb5||strngb6||
        strngb7||strngb8||strngb9||strngb10||strngb11||strngb12||
        strngb13;
    CCATOF=strngc1||strngc2||strngc3||strngc4||strngc5||strngc6||
        strngc7||strngc8||strngc9||strngc10;

   label CCAT23F="concatenation of code 23 labor ?s";
   label CCATVF="concatenation of village labor ?s";
   label CCATOF="concatenation of oth vil labor ?s";

run;


/* proc freq data=work00_4;
     tables CCATALLA CCAT23F CCATVF CCATOF/ missprint;
run; */


proc datasets library=work;
     delete work00_3;
run;


data work00_5 (drop=HELPVH_1 HELPVH_2 HELPVH_3
                    HELPOH_1 HELPOH_2 HELPOH_3 i j);
     set work00_4;

     array vf(1:13) HELPVF1-HELPVF13;
     array of(1:10) HELPOF1-HELPOF10;

     HELPVH_1=0;
     HELPVH_2=0;
     HELPVH_3=0;
     HELPOH_1=0;
     HELPOH_2=0;
     HELPOH_3=0;

     do i=1 to 13;
        if vf(i)=1 then HELPVH_1=HELPVH_1+1;
           else if vf(i)=2 then HELPVH_2=HELPVH_2+1;
           else if vf(i)=3 then HELPVH_3=HELPVH_3+1;
     end;

     do j=1 to 10;
        if of(j)=1 then HELPOH_1=HELPOH_1+1;
           else if of(j)=2 then HELPOH_2=HELPOH_2+1;
           else if of(j)=3 then HELPOH_3=HELPOH_3+1;
     end;

     if HELPVH_1>0 then HELPVH=1;
        else if HELPVH_2>0 | HELPVH_3>0 then HELPVH=2;
                              else HELPVH=.;
     if HELPOH_1>0 then HELPOH=1;
        else if HELPOH_2>0 | HELPOH_3>0 then HELPOH=2;
                              else HELPOH=.;

     label HELPVH= 'Used village labor 1=paid 2=unpaid';
     label HELPOH= 'Used non-village labor 1=paid 2=unpaid';

     if RICE=0 then HELPDV=1;
        else if RICE=. then HELPDV=.;
        else if HELP23A=0 & HELPVA=0 & HELPOA=0 then HELPDV=2;
        else if HELP23A=1 & HELPVA=0 & HELPOA=0 then HELPDV=3;
        else if (HELPVH ne 1 & HELPOH ne 1) & (HELPVA=1 or HELPOA=1) then HELPDV=4;
        else if HELPVH=1 & HELPOH ne 1 then HELPDV=5;
        else if HELPOH=1 & HELPVH ne 1 then HELPDV=6;
        else if HELPVH=1 & HELPOH=1 then HELPDV=7;

     if RICE=0 then HELPDV2=1;
        else if RICE=. then HELPDV2=.;
        else if HELP23A=0 & HELPVA=0 & HELPOA=0 then HELPDV2=2;
        else if HELP23A=1 & HELPVA=0 & HELPOA=0 then HELPDV2=3;
        else if (HELPVH ne 1 & HELPOH ne 1) & (HELPVA=1 or HELPOA=1) then HELPDV2=3;
        else if HELPVH=1 & HELPOH ne 1 then HELPDV2=4;
        else if HELPOH=1 & HELPVH ne 1 then HELPDV2=4;
        else if HELPVH=1 & HELPOH=1 then HELPDV2=4;
run;

/* proc freq data=work00_5;
     tables HELPVH HELPOH HELPDV HELPDV2;
run; */

proc sort data=work00_5 out=sorted94;
     by VILL94;
run;

data vill_id_fix;
     set extra_1.hh94;
     keep VILL94 VILL84;
run;

proc sort data=vill_id_fix out=vill_id_fix2 nodupkey;
     by VILL94;
run;

data vill_id_fix3;
     merge sorted94 (in=a drop=VILL84)
           vill_id_fix2 (in=b);
     by VILL94;
run;

proc sort data=vill_id_fix3 out=sorted84;
     by VILL84;
run;

proc freq data=sorted84;
     tables VILL84*HELPDV2/ NOPERCENT NOCOL NOFREQ;
run;

/* proc freq data=work00_5;
   tables VILL94*HELPDV VILL94*HELPDV2/ NOPERCENT NOCOL NOFREQ;
run; */

proc contents data=work00_5 varnum;
run;

data out00_1.help0001;
     set work00_5;
run;

/* proc datasets library=work;
     delete work00_4 work00_5;
run; */


### MIgration, Remittances, and Monetization of Farm Labor in Subsistence Sending Areas (Master's Thesis)

This notebook documents the code I used in producing the paper located here (my Master's Thesis):
 
Hull JR. Migration, Remittances and Monetization of Farm Labor in Subsistence Sending Areas. Asian and Pacific Migration Journal. 2007;16(4):451-484. doi:10.1177/011719680701600402

Code is presented untouched for archival purposes, with some explanatory markup and reflections added.
  


In [ ]:
*********************************************************************
**     Program Name: /trainee/jrhull/ma/migm9400.sas
**     Programmer: james r. hull
**     Start Date: 2006 January 21
**     Purpose:
**        1.) Match-merge 1994 and 2000 data sets
**     Input Data:
**        2.) /trainee/jrhull/ma/merge94.xpt
**        4.) /trainee/jrhull/ma/merge00.xpt
**
*********************************************************************;

*--------------*
*   Options    *
*--------------*;

options nocenter linesize=80 pagesize=60;

*------------------*
*  Data Libraries  *
*------------------*;

libname in94_1 xport '/trainee/jrhull/ma/merge94.xpt' ;
libname in00_1 xport '/trainee/jrhull/ma/merge00.xpt' ;

libname out1 xport '/trainee/jrhull/ma/merg9400.xpt';

*-------------------------------------------*
* Generate pre-merge descriptive statistics *
*-------------------------------------------*;

proc freq data=in94_1.merge94;
     tables hhtype94;
     /* tables (numrcd94 numsnd94)*nummig94 / nopercent norow nocol;  */
run;

proc freq data=in00_1.merge00;
     tables hhtype00;
     /* tables (numrcd00 numsnd00)*nummig00 / nopercent norow nocol;  */
run;

*------------------------------*
* Merge 1994 and 2000 datasets *
*------------------------------*;

proc sort data=in00_1.merge00 out=merge00;
     by vill94 hhid94;
run;

data merg9400 no94data no00data;
     merge in94_1.merge94 (in=a)
           merge00 (in=b);
     by vill94 hhid94;

     if a=1 and b=1 then output merg9400;
     if a=1 and b=0 then output no00data;
     if a=0 and b=1 then output no94data;
run;

proc freq data=merg9400;
     tables hhtype94 hhtype00;
     /* tables (code2_00 nummig00)*hhtype94; */
     tables helpd294*helpd200 helpd194*helpd100;
run;

data out1.merg9400;
     set merg9400;
run;

data merg94_2;
     set in94_1.merge94;
     if hhtype94 in (1,3);
run;

data merg00_2;
     set merge00;
     if hhtype00 in (1,3);
run;

proc freq data=merg94_2;
     tables hhtype94;
run;

proc freq data=merg00_2;
     tables hhtype00;
run;

data mergdat2 no94_2 no00_2;
     merge merg94_2 (in=a)
           merg00_2 (in=b);
     by vill94 hhid94;

     if a=1 and b=1 then output mergdat2;
     if a=1 and b=0 then output no00_2;
     if a=0 and b=1 then output no94_2;

run;

proc freq data=mergdat2;
     tables hhtype94 hhtype00;
     /* tables (code2_00 nummig00)*hhtype94; */
     tables helpd294*helpd200 helpd194*helpd100;
run;


In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_01.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2005 October 14                            */
/*                                                  */ 
/* PURPOSE: GENERATE MNL MODELS FOR MA              */


/* 1994 DATA */

capture log close
log using reg94_01, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Descriptive Work */

/* histogram logmnage 
histogram meanage
histogram cows
histogram pigs */


/* Simplest model */

mlogit helpdv2 nummigt, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 nummigt, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 nummigt, robust cluster(vill94) basecategory(2)          



/* One-variable model - number of remitters */

mlogit helpdv2 numrrcd2, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numrrcd2, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numrrcd2, robust cluster(vill94) basecategory(2)          



* One-variable model - number of migrants receiving money */

mlogit helpdv2 numrsnd2, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numrsnd2, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numrsnd2, robust cluster(vill94) basecategory(2)          

 
/* Full Model - Revised Remittance Variables */

mlogit helpdv2 nummigt numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigt numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigt numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - male and female migrants separate */

mlogit helpdv2 nummigm nummigf numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigm nummigf numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigm nummigf numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Collinearity Check */

reg helpdv2 nummigt numrrcd2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai

vif


/* Full Model - Number migrants only */

mlogit helpdv2 nummigt m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigt m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigt m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number remitters (new variable) only */

mlogit helpdv2 numrrcd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrrcd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrrcd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number sent (new variable) only */

mlogit helpdv2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrsnd2 m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_02.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 January 21                            */
/*                                                  */
/* PURPOSE: TEST FUNCTIONAL FORM OF KEY IV's        */


/* 1994 DATA */

capture log close
log using reg94_02, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Descriptive Work */

/* histogram logmnage 
histogram meanage
histogram cows
histogram pigs */

/* Create categorical variables for # Migrants & # Remitters */

gen nummig0=nummigt
gen nummig1=nummigt
gen nummig2=nummigt
gen nummig3p=nummigt

recode nummig0 (0=1) (1/9=0)
recode nummig1 (2/9=0)
recode nummig2 (1=0) (2=1) (3/9=0)
recode nummig3p (1/2=0) (3/9=1)

gen numrcd0=numrrcd2
gen numrcd1=numrrcd2
gen numrcd2=numrrcd2
gen numrcd3p=numrrcd2

recode numrcd0 (0=1) (1/7=0)
recode numrcd1 (2/7=0)
recode numrcd2 (1=0) (2=1) (3/7=0)
recode numrcd3p (1/2=0) (3/7=1)

gen numsnd0=numrsnd2
gen numsnd1=numrsnd2
gen numsnd2=numrsnd2
gen numsnd3p=numrsnd2

recode numsnd0 (0=1) (1/5=0)
recode numsnd1 (2/5=0)
recode numsnd2 (1=0) (2=1) (3/5=0)
recode numsnd3p (1/2=0) (3/5=1)



/* Simplest model */

mlogit helpdv2 nummig1 nummig2 nummig3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 nummig1 nummig2 nummig3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 nummig1 nummig2 nummig3p, robust cluster(vill94) basecategory(2) 


mlogit helpdv2 nummig0 nummig2 nummig3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 nummig0 nummig2 nummig3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 nummig0 nummig2 nummig3p, robust cluster(vill94) basecategory(2) 


mlogit helpdv2 nummig0 nummig1 nummig3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 nummig0 nummig1 nummig3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 nummig0 nummig1 nummig3p, robust cluster(vill94) basecategory(2) 



/* One-variable model - number of remitters */

mlogit helpdv2 numrcd1 numrcd2 numrcd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numrcd1 numrcd2 numrcd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numrcd1 numrcd2 numrcd3p, robust cluster(vill94) basecategory(2)          


mlogit helpdv2 numrcd0 numrcd2 numrcd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numrcd0 numrcd2 numrcd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numrcd0 numrcd2 numrcd3p, robust cluster(vill94) basecategory(2)          


mlogit helpdv2 numrcd0 numrcd1 numrcd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numrcd0 numrcd1 numrcd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numrcd0 numrcd1 numrcd3p, robust cluster(vill94) basecategory(2)          



* One-variable model - number of migrants receiving money */

mlogit helpdv2 numsnd1 numsnd2 numsnd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numsnd1 numsnd2 numsnd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numsnd1 numsnd2 numsnd3p, robust cluster(vill94) basecategory(2)          


mlogit helpdv2 numsnd0 numsnd2 numsnd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numsnd0 numsnd2 numsnd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numsnd0 numsnd2 numsnd3p, robust cluster(vill94) basecategory(2)          


mlogit helpdv2 numsnd0 numsnd1 numsnd3p, robust cluster(vill94) basecategory(4)          

mlogit helpdv2 numsnd0 numsnd1 numsnd3p, robust cluster(vill94) basecategory(3)          

mlogit helpdv2 numsnd0 numsnd1 numsnd3p, robust cluster(vill94) basecategory(2)          

 
 

 
/* Full Model - Number migrants only */

mlogit helpdv2 nummig1 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummig1 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummig1 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 nummig0 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummig0 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummig0 nummig2 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 nummig0 nummig1 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummig0 nummig1 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummig0 nummig1 nummig3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



/* Full Model - Number remitters (new variable) only */

mlogit helpdv2 numrcd1 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrcd1 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrcd1 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 numrcd0 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrcd0 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrcd0 numrcd2 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 numrcd0 numrcd1 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrcd0 numrcd1 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrcd0 numrcd1 numrcd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



/* Full Model - Number sent (new variable) only */

mlogit helpdv2 numsnd1 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numsnd1 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numsnd1 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 numsnd0 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numsnd0 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numsnd0 numsnd2 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


mlogit helpdv2 numsnd0 numsnd1 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numsnd0 numsnd1 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numsnd0 numsnd1 numsnd3p m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)




         


In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_03.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 January 24                            */
/*                                                  */ 
/* PURPOSE: GENERATE MNL MODELS FOR MA (W/CODETWO)  */


/* 1994 DATA */

capture log close
log using reg94_03, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Descriptive Work */

/* histogram logmnage 
histogram meanage
histogram cows
histogram pigs */
 

summarize helpdv2 nummigt numrrcd2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs0 pigs1 cows0 cows1_6 cows7pls charcoal lowest middle highest vill1365 vill_rai

 
/* Collinearity Check */

reg helpdv2 nummigt numrrcd2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai

vif


/* Full Model - Number migrants only */

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number remitters (new variable) only */

mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number sent (new variable) only */

mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



In [ ]:
* PROGRAM NAME: /trainee/jrhull/ma/reg94_04.do      */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 January 25                            */
/*                                                  */ 
/* PURPOSE: GENERATE MN PROBIT MODELS FOR MA        */


/* 1994 DATA */

capture log close
log using reg94_04, replace text

use help9403.dta, replace

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)



/* Simplest model */

mprobit helpdv2 nummigt, robust baseoutcome(4)          

mprobit helpdv2 nummigt, robust baseoutcome(3)          

mprobit helpdv2 nummigt, robust baseoutcome(2)          



/* One-variable model - number of remitters */

mprobit helpdv2 numrrcd2, robust baseoutcome(4)          

mprobit helpdv2 numrrcd2, robust baseoutcome(3)          

mprobit helpdv2 numrrcd2, robust baseoutcome(2)          



* One-variable model - number of migrants receiving money */

mprobit helpdv2 numrsnd2, robust baseoutcome(4)          

mprobit helpdv2 numrsnd2, robust baseoutcome(3)          

mprobit helpdv2 numrsnd2, robust baseoutcome(2)          



/* Full Model - Number migrants only */

mprobit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


/* Full Model - Number remitters (new variable) only */

mprobit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


/* Full Model - Number sent (new variable) only */

mprobit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_05.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 January 26                            */
/*                                                  */ 
/* PURPOSE: GENERATE MNL MODELS FOR MA (5-Category) */
 

/* 1994 DATA */

capture log close
log using reg94_05, replace text

use help9403.dta, replace

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Descriptive Work */

/* histogram logmnage 
histogram meanage
histogram cows
histogram pigs */

gen helpdv3=helpdv
recode helpdv3 (1=1) (2=2) (3/4=3) (5=4) (6/7=5)


/* Full Model - Number migrants only */

mlogit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(5)

mlogit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

mlogtest, combine



/* Full Model - Number remitters (new variable) only */

mlogit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(5)

mlogit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv3 numrrcd2 codetwo  m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

mlogtest, combine



/* Full Model - Number sent (new variable) only */


mlogit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(5)

mlogit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

mlogtest, combine





In [ ]:
* PROGRAM NAME: /trainee/jrhull/ma/reg94_06.do      */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 January 26                            */
/*                                                  */ 
/* PURPOSE: GENERATE MN PROBIT MODELS FOR MA (5-cat)*/


/* 1994 DATA */

capture log close
log using reg94_06, replace text

use help9403.dta, replace

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

gen helpdv3=helpdv
recode helpdv3 (1=1) (2=2) (3/4=3) (5=4) (6/7=5)


/* Simplest model */

mprobit helpdv3 nummigt, robust baseoutcome(5)

mprobit helpdv3 nummigt, robust baseoutcome(4)          

mprobit helpdv3 nummigt, robust baseoutcome(3)          

mprobit helpdv3 nummigt, robust baseoutcome(2)          



/* One-variable model - number of remitters */

mprobit helpdv3 numrrcd2, robust baseoutcome(5) 

mprobit helpdv3 numrrcd2, robust baseoutcome(4)          

mprobit helpdv3 numrrcd2, robust baseoutcome(3)          

mprobit helpdv3 numrrcd2, robust baseoutcome(2)          



* One-variable model - number of migrants receiving money */

mprobit helpdv3 numrsnd2, robust baseoutcome(5)  

mprobit helpdv3 numrsnd2, robust baseoutcome(4)          

mprobit helpdv3 numrsnd2, robust baseoutcome(3)          

mprobit helpdv3 numrsnd2, robust baseoutcome(2)          



/* Full Model - Number migrants only */

mprobit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(5)

mprobit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv3 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


/* Full Model - Number remitters (new variable) only */


mprobit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(5)

mprobit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv3 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


/* Full Model - Number sent (new variable) only */


mprobit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(5)

mprobit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(4)

mprobit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(3)

mprobit helpdv3 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust baseoutcome(2)


In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_07.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 FEBRUARY 6                            */
/*                                                  */ 
/* PURPOSE: PREDICTED PROB'S 4-CATEGORY W/CODETWO   */


/* 1994 DATA */

capture log close
log using reg94_07, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)


/* MODEL I */

/* Full Model - Number migrants only */

quietly mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

quietly mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

quietly mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* PREDICTED PROB - MODEL I, ALL VARIABLES TAKE ACTUAL DATA VALUES */

quietly predict p1m p2m p3m p4m, pr


/* PREDICTED PROB - MODEL I, NUMMIGT=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdm1_00=(exp(0*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(0*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm2_00=(exp(0*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(0*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm3_00=(exp(0*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(0*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm4_00=(exp(0*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(0*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - MODEL I, NUMMIGT=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdm1_01=(exp(1*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(1*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm2_01=(exp(1*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(1*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm3_01=(exp(1*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(1*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm4_01=(exp(1*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(1*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL I, NUMMIGT=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdm1_02=(exp(2*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(2*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm2_02=(exp(2*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(2*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm3_02=(exp(2*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(2*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm4_02=(exp(2*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(2*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL I, NUMMIGT=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdm1_03=(exp(3*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(3*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm2_03=(exp(3*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(3*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm3_03=(exp(3*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(3*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm4_03=(exp(3*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(3*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL I, NUMMIGT=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdm1_04=(exp(4*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(4*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm2_04=(exp(4*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(4*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm3_04=(exp(4*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(4*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdm4_04=(exp(4*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(4*[1]_b[nummigt]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[nummigt]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[nummigt]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[nummigt]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* MODEL II */

/* Full Model - Number remitters (new variable) only */

quietly mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

quietly mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

quietly mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* PREDICTED PROB - MODEL II,ALL VARIABLES TAKE ACTUAL DATA VALUES */

quietly predict p1r p2r p3r p4r, pr


/* PREDICTED PROB - MODEL II, NUMRCD2=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdr1_00=(exp(0*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(0*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr2_00=(exp(0*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(0*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr3_00=(exp(0*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(0*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr4_00=(exp(0*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(0*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL II, NUMRCD2=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdr1_01=(exp(1*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(1*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr2_01=(exp(1*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(1*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr3_01=(exp(1*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(1*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr4_01=(exp(1*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(1*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL II, NUMRCD2=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdr1_02=(exp(2*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(2*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr2_02=(exp(2*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(2*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr3_02=(exp(2*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(2*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr4_02=(exp(2*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(2*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL II, NUMRCD2=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdr1_03=(exp(3*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(3*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr2_03=(exp(3*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(3*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr3_03=(exp(3*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(3*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr4_03=(exp(3*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(3*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL II, NUMRCD2=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdr1_04=(exp(4*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(4*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr2_04=(exp(4*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(4*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr3_04=(exp(4*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(4*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdr4_04=(exp(4*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(4*[1]_b[numrrcd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrrcd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrrcd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrrcd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* MODEL III */

/* Full Model - Number sent (new variable) only */

quietly mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

quietly mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

quietly mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* PREDICTED PROB - MODEL III, ALL VARIABLES TAKE ACTUAL DATA VALUES */

quietly predict p1s p2s p3s p4s, pr


/* PREDICTED PROB - MODEL III, NUMSND2=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prds1_00=(exp(0*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(0*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds2_00=(exp(0*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(0*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds3_00=(exp(0*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(numrsnd2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(numrsnd2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(numrsnd2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(numrsnd2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds4_00=(exp(0*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(0*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL III, NUMSND2=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prds1_01=(exp(1*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(1*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds2_01=(exp(1*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(1*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds3_01=(exp(1*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(1*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds4_01=(exp(1*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(1*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL III, NUMSND2=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prds1_02=(exp(2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds2_02=(exp(2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds3_02=(exp(2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds4_02=(exp(2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(2*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL III, NUMSND2=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prds1_03=(exp(3*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(3*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds2_03=(exp(3*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(3*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds3_03=(exp(3*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(3*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds4_03=(exp(3*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(3*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - MODEL III, NUMSND2=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prds1_04=(exp(4*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(4*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds2_04=(exp(4*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(4*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds3_04=(exp(4*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(4*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prds4_04=(exp(4*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(4*[1]_b[numrsnd2]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[numrsnd2]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[numrsnd2]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[numrsnd2]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

summarize p1m prdm1_00 prdm1_01 prdm1_02 prdm1_03 prdm1_04 p2m prdm2_00 prdm2_01 prdm2_02 prdm2_03 prdm2_04 p3m prdm3_00 prdm3_01 prdm3_02 prdm3_03 prdm3_04 p4m prdm4_00 prdm4_01 prdm4_02 prdm4_03 prdm4_04
summarize p1r prdr1_00 prdr1_01 prdr1_02 prdr1_03 prdr1_04 p2r prdr2_00 prdr2_01 prdr2_02 prdr2_03 prdr2_04 p3r prdr3_00 prdr3_01 prdr3_02 prdr3_03 prdr3_04 p4r prdr4_00 prdr4_01 prdr4_02 prdr4_03 prdr4_04 
summarize p1s prds1_00 prds1_01 prds1_02 prds1_03 prds1_04 p2s prds2_00 prds2_01 prds2_02 prds2_03 prds2_04 p3s prds3_00 prds3_01 prds3_02 prds3_03 prds3_04 p4s prds4_00 prds4_01 prds4_02 prds4_03 prds4_04





In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_08.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 February 18                           */
/*                                                  */
/* PURPOSE: TEST ALTERNATE SPECIFICATIONS OF IV'S   */
/*          USING 3-year WINDOW (NOT 6-year)        */


/* 1994 DATA */

capture log close
log using reg94_08, replace text

use help9403.dta, replace

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Descriptive Analysis */


summarize helpdv2 nummigt2 numrrcd3 numrsnd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs0 pigs1 cows0 cows1_6 cows7pls charcoal lowest middle highest vill1365 vill_rai

 
/* Collinearity Check */

reg helpdv2 nummigt2 numrrcd3 numrsnd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai

vif


/* Full Model - Number migrants only */

mlogit helpdv2 nummigt2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigt2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigt2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number remitters (new variable) only */

mlogit helpdv2 numrrcd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrrcd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrrcd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number sent (new variable) only */

mlogit helpdv2 numrsnd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numrsnd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numrsnd3 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_09.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 February 18                           */
/*                                                  */
/* PURPOSE: TEST IIA ASSUMPTION                     */


/* 1994 DATA */

capture log close
log using reg94_09, replace text

use help9403.dta, replace

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)

/* Full Model - Number migrants only */

quietly: mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

set seed 1010101

mlogtest, smhsiao

quietly: mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

set seed 1010102

mlogtest, smhsiao

quietly: mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

set seed 1010103

mlogtest, smhsiao

mlogtest, hausman base


/* Full Model - Number remitters (new variable) only */

quietly: mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

set seed 1010104

mlogtest, smhsiao

quietly: mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)


set seed 1010105

mlogtest, smhsiao

quietly: mlogit helpdv2 numrrcd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

set seed 1010106

mlogtest, smhsiao

mlogtest, hausman base

/* Full Model - Number sent (new variable) only */

quietly: mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

set seed 1010107

mlogtest, smhsiao

quietly: mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)


set seed 1010108

mlogtest, smhsiao

quietly: mlogit helpdv2 numrsnd2 codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)

set seed 1010109

mlogtest, smhsiao

mlogtest, hausman base



In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_10.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 February 28                           */
/*                                                  */ 
/* PURPOSE: GENERATE MNL MODELS FOR MA (W/CODETWO)  */
/*          USING TIME-UNLIMITED REMITTANCE VAR'S   */

/* 1994 DATA */

capture log close
log using reg94_10, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)


/* Descriptives */

summarize helpdv2 nummigt numremit numsend codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs0 pigs1 cows0 cows1_6 cows7pls charcoal lowest middle highest vill1365 vill_rai

 
/* Collinearity Check */

reg helpdv2 nummigt numremit numsend codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai

vif


/* Full Model - Number migrants only */

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 nummigt codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number remitters (new variable) only */

mlogit helpdv2 numremit codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numremit codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numremit codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)


/* Full Model - Number sent (new variable) only */

mlogit helpdv2 numsend codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 numsend codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 numsend codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



In [ ]:
/* PROGRAM NAME: /trainee/jrhull/ma/reg94_11.do     */
/* PROGRAMER: jrhull                                */
/* DATE: 2006 June 26                               */
/*                                                  */ 
/* PURPOSE: GENERATE MNL MODELS with new Dep Var    */


/* 1994 DATA */

capture log close
log using reg94_11, replace text

use help9403.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/122=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/122=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/122=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/110=0) (.=.)
recode pigs1 (0=0) (1/110=1) (.=.)


/* Descriptive Work */

/* histogram migrem_y 
histogram migrem_n */


summarize helpdv2 migrem_y migrem_n codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs0 pigs1 cows0 cows1_6 cows7pls charcoal lowest middle highest vill1365 vill_rai


/* Collinearity Check */

reg helpdv2 migrem_y migrem_n codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai

vif


/* Full Model - New Combined Variables */

mlogit helpdv2 migrem_y migrem_n codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(4)

mlogit helpdv2 migrem_y migrem_n codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(3)

mlogit helpdv2 migrem_y migrem_n codetwo m_13_65 f_13_65 numdeps meanage cassava cottage pigs1 cows1_6 cows7pls charcoal middle highest vill1365 vill_rai, robust cluster(vill94) basecategory(2)



/* PREDICTED PROB - FULL MODEL,ALL VARIABLES TAKE ACTUAL DATA VALUES */

quietly predict p1y p2y p3y p4y, pr



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdy1_00=(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy2_00=(exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy3_00=(exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy4_00=(exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdy1_01=(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy2_01=(exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy3_01=(exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy4_01=(exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdy1_02=(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy2_02=(exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy3_02=(exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy4_02=(exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdy1_03=(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy2_03=(exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy3_03=(exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy4_03=(exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdy1_04=(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy2_04=(exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy3_04=(exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdy4_04=(exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - MODEL II,ALL VARIABLES TAKE ACTUAL DATA VALUES */

quietly predict p1n p2n p3n p4n, pr


/* PREDICTED PROB - FULL MODEL, MIGREM_N=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdn1_00=(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn2_00=(exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn3_00=(exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn4_00=(exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_N=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdn1_01=(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn2_01=(exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn3_01=(exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn4_01=(exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdn1_02=(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn2_02=(exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn3_02=(exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn4_02=(exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdn1_03=(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn2_03=(exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn3_03=(exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn4_03=(exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES */

quietly gen prdn1_04=(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn2_04=(exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn3_04=(exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdn4_04=(exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


summarize p1y prdy1_00 prdy1_01 prdy1_02 prdy1_03 prdy1_04 p2y prdy2_00 prdy2_01 prdy2_02 prdy2_03 prdy2_04 p3y prdy3_00 prdy3_01 prdy3_02 prdy3_03 prdy3_04 p4y prdy4_00 prdy4_01 prdy4_02 prdy4_03 prdy4_04

summarize p1n prdn1_00 prdn1_01 prdn1_02 prdn1_03 prdn1_04 p2n prdn2_00 prdn2_01 prdn2_02 prdn2_03 prdn2_04 p3n prdn3_00 prdn3_01 prdn3_02 prdn3_03 prdn3_04 p4n prdn4_00 prdn4_01 prdn4_02 prdn4_03 prdn4_04 



/* STANDARDIZATION ACROSS YEARS - 2000 DATA/1994 COEFF'S */

use help0003.dta, replace

gen logmnage=log(meanage)

gen cows0=cows
gen cows1_6=cows
gen cows7pls=cows
recode cows0 (0=1) (1/6=0) (7/312=0) (.=.)
recode cows1_6 (0=0) (1/6=1) (7/312=0) (.=.)
recode cows7pls (0=0) (1/6=0) (7/312=1) (.=.)

gen pigs0=pigs
gen pigs1=pigs
recode pigs0 (0=1) (1/800=0) (9999=.) (.=.)
recode pigs1 (0=0) (1/800=1) (9999=.) (.=.)

/* PREDICTED PROB - FULL MODEL,ALL VARIABLES TAKE ACTUAL DATA VALUES  - 2000 DATA/1994 COEFF'S */

quietly predict p1e p2e p3e p4e, pr


/* PREDICTED PROB - FULL MODEL, MIGREM_Y=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prde1_00=(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde2_00=(exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde3_00=(exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde4_00=(exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(0*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(0*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(0*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(0*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prde1_01=(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde2_01=(exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde3_01=(exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde4_01=(exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(1*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(1*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(1*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(1*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prde1_02=(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde2_02=(exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde3_02=(exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde4_02=(exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(2*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(2*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(2*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(2*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prde1_03=(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde2_03=(exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde3_03=(exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde4_03=(exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(3*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(3*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(3*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(3*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_Y=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prde1_04=(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde2_04=(exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde3_04=(exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prde4_04=(exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(4*[1]_b[migrem_y]+migrem_n*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(4*[2]_b[migrem_y]+migrem_n*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(4*[3]_b[migrem_y]+migrem_n*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(4*[4]_b[migrem_y]+migrem_n*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL,ALL VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly predict p1o p2o p3o p4o, pr


/* PREDICTED PROB - FULL MODEL, MIGREM_N=0, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prdo1_00=(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo2_00=(exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo3_00=(exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo4_00=(exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+0*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+0*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+0*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+0*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))



/* PREDICTED PROB - FULL MODEL, MIGREM_N=1, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prdo1_01=(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo2_01=(exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo3_01=(exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo4_01=(exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+1*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+1*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+1*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+1*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=2, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prdo1_02=(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo2_02=(exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo3_02=(exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo4_02=(exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+2*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+2*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+2*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+2*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=3, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prdo1_03=(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo2_03=(exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo3_03=(exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo4_03=(exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+3*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+3*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+3*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+3*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))


/* PREDICTED PROB - FULL MODEL, MIGREM_N=4, ALL OTHER VARIABLES TAKE ACTUAL DATA VALUES - 2000 DATA/1994 COEFF'S */

quietly gen prdo1_04=(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo2_04=(exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo3_04=(exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

quietly gen prdo4_04=(exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[4]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))/(exp(migrem_y*[1]_b[migrem_y]+4*[1]_b[migrem_n]+codetwo*[1]_b[codetwo]+m_13_65*[1]_b[m_13_65]+f_13_65*[1]_b[f_13_65]+numdeps*[1]_b[numdeps]+meanage*[1]_b[meanage]+cassava*[1]_b[cassava]+cottage*[1]_b[cottage]+pigs1*[1]_b[pigs1]+cows1_6*[1]_b[cows1_6]+cows7pls*[1]_b[cows7pls]+charcoal*[1]_b[charcoal]+middle*[1]_b[middle]+highest*[1]_b[highest]+vill1365*[1]_b[vill1365]+vill_rai*[1]_b[vill_rai]+[1]_b[_cons])+exp(migrem_y*[2]_b[migrem_y]+4*[2]_b[migrem_n]+codetwo*[2]_b[codetwo]+m_13_65*[2]_b[m_13_65]+f_13_65*[2]_b[f_13_65]+numdeps*[2]_b[numdeps]+meanage*[2]_b[meanage]+cassava*[2]_b[cassava]+cottage*[2]_b[cottage]+pigs1*[2]_b[pigs1]+cows1_6*[2]_b[cows1_6]+cows7pls*[2]_b[cows7pls]+charcoal*[2]_b[charcoal]+middle*[2]_b[middle]+highest*[2]_b[highest]+vill1365*[2]_b[vill1365]+vill_rai*[2]_b[vill_rai]+[2]_b[_cons])+exp(migrem_y*[3]_b[migrem_y]+4*[3]_b[migrem_n]+codetwo*[3]_b[codetwo]+m_13_65*[3]_b[m_13_65]+f_13_65*[3]_b[f_13_65]+numdeps*[3]_b[numdeps]+meanage*[3]_b[meanage]+cassava*[3]_b[cassava]+cottage*[3]_b[cottage]+pigs1*[3]_b[pigs1]+cows1_6*[3]_b[cows1_6]+cows7pls*[3]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[3]_b[middle]+highest*[3]_b[highest]+vill1365*[3]_b[vill1365]+vill_rai*[3]_b[vill_rai]+[3]_b[_cons])+exp(migrem_y*[4]_b[migrem_y]+4*[4]_b[migrem_n]+codetwo*[4]_b[codetwo]+m_13_65*[4]_b[m_13_65]+f_13_65*[4]_b[f_13_65]+numdeps*[4]_b[numdeps]+meanage*[4]_b[meanage]+cassava*[4]_b[cassava]+cottage*[4]_b[cottage]+pigs1*[4]_b[pigs1]+cows1_6*[4]_b[cows1_6]+cows7pls*[4]_b[cows7pls]+charcoal*[3]_b[charcoal]+middle*[4]_b[middle]+highest*[4]_b[highest]+vill1365*[4]_b[vill1365]+vill_rai*[4]_b[vill_rai]+[4]_b[_cons]))

summarize p1e prde1_00 prde1_01 prde1_02 prde1_03 prde1_04 p2e prde2_00 prde2_01 prde2_02 prde2_03 prde2_04 p3e prde3_00 prde3_01 prde3_02 prde3_03 prde3_04 p4e prde4_00 prde4_01 prde4_02 prde4_03 prde4_04

summarize p1o prdo1_00 prdo1_01 prdo1_02 prdo1_03 prdo1_04 p2o prdo2_00 prdo2_01 prdo2_02 prdo2_03 prdo2_04 p3o prdo3_00 prdo3_01 prdo3_02 prdo3_03 prdo3_04 p4o prdo4_00 prdo4_01 prdo4_02 prdo4_03 prdo4_04 

